In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Load requirements

In [38]:
import pandas as pd
import pathlib
import matplotlib as pyplt
import cv2
import os
import ssl
import traceback
import logging
from tqdm import tqdm
import urllib
import ipdb

from sklearn.cluster import KMeans
import numpy as np
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76

In [3]:
#DATA_DIR = os.path.join(os.path.dirname ("__file__"), os.path.pardir)

#os.chdir(DATA_DIR)
# print(f"the root directory is {ROOT_DIR}")
# print(f"the data directory is {DATA_DIR}")
# os.chdir(DATA_DIR)
# !ls

In [20]:
def get_data():
    # use a subset of data from WikiArt for training
    !curl -O http://saifmohammad.com/WebDocs/WikiArt-Emotions.zip
    !unzip -o WikiArt-Emotions.zip
    !rm WikiArt-Emotions.zip
    !mv WikiArt-Emotions ../raw_data/WikiArt-Emotions
    !ls
    ROOT_DIR = os.path.join(os.path.dirname ("__file__"), os.path.pardir)
    DATA_DIR = os.path.join(ROOT_DIR, "raw_data")
    os.chdir(DATA_DIR)
    PATH = pathlib.Path('WikiArt-Emotions')
    !ls
    df = pd.read_csv('WikiArt-Emotions/WikiArt-info.tsv', delimiter='\t'); df.head()
    return df, DATA_DIR, PATH

In [21]:
df = get_data()[0]
DATA_DIR = get_data()[1]
PATH = get_data()[2]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1901k  100 1901k    0     0   619k      0  0:00:03  0:00:03 --:--:--  619k
Archive:  WikiArt-Emotions.zip
  inflating: __MACOSX/._WikiArt-Emotions  
  inflating: WikiArt-Emotions/WikiArt-Emotions-All.tsv  
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-Emotions-All.tsv  
  inflating: WikiArt-Emotions/WikiArt-annotations.csv  
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-annotations.csv  
  inflating: WikiArt-Emotions/WikiArt-info.tsv  
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-info.tsv  
  inflating: WikiArt-Emotions/README.txt  
  inflating: __MACOSX/WikiArt-Emotions/._README.txt  
  inflating: WikiArt-Emotions/WikiArt-Emotions-Ag5.tsv  
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-Emotions-Ag5.tsv  
  inflating: WikiArt-Emotions/WikiArt-Emotions-Ag4.tsv  
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-Emot

In [22]:
#df.head()
PATH

PosixPath('WikiArt-Emotions')

In [29]:
urls = df['Image URL'].tolist(); urls[:5]

['https://uploads3.wikiart.org/00123/images/charles-courtney-curran/in-the-luxembourg-garden-1889.jpg',
 'https://uploads1.wikiart.org/images/keith-haring/the-marriage-of-heaven-and-hell-1984.jpg',
 'https://uploads3.wikiart.org/images/j-zsef-rippl-r-nai/uncle-piacsek-in-front-of-the-black-sideboard-1906.jpg',
 'https://uploads2.wikiart.org/00124/images/vadym-meller/monk-for-the-play-mazeppa-1920.jpg',
 'https://uploads6.wikiart.org/images/david-wilkie/the-defence-of-sarago-a.jpg']

## Download images

In [37]:
IMG_DIR = PATH/'images'
IMG_DIR.mkdir(parents=True, exist_ok=True)
print(IMG_DIR)

WikiArt-Emotions/images


In [39]:
# ignore SSL errors
ssl._create_default_https_context = ssl._create_unverified_context

def download(path, urls):
    for url in tqdm(urls):
        try:
            fname = url[url.rfind('/')+1:]
            urllib.request.urlretrieve(url, path/fname)
        except Exception as e:
            logging.error(traceback.format_exc())

In [40]:
#ipdb.set_trace()
download(IMG_DIR, urls)

  2%|█▊                                                                             | 92/4119 [02:01<1:23:48,  1.25s/it]ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_1884/2130755272.py", line 8, in download
    urllib.request.urlretrieve(url, path/fname)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 247, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 542, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = fu

 16%|████████████▏                                                                 | 642/4119 [15:03<1:17:28,  1.34s/it]ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_1884/2130755272.py", line 8, in download
    urllib.request.urlretrieve(url, path/fname)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 247, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 542, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = fu

 32%|█████████████████████████▋                                                     | 1337/4119 [31:43<59:04,  1.27s/it]ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_1884/2130755272.py", line 8, in download
    urllib.request.urlretrieve(url, path/fname)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 247, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 542, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = fu

 52%|█████████████████████████████████████████▎                                     | 2156/4119 [51:12<57:09,  1.75s/it]ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_1884/2130755272.py", line 8, in download
    urllib.request.urlretrieve(url, path/fname)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 247, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 542, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/home/erniesg/.pyenv/versions/3.8.12/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = fu

# Extract colours from the paintings

In [41]:
image = cv2.imread('sample_image.jpg')
#image = io.imread('https://uploads3.wikiart.org/00123/images/charles-courtney-curran/in-the-luxembourg-garden-1889.jpg')
#image = cv2.imread('https://uploads3.wikiart.org/00123/images/charles-courtney-curran/in-the-luxembourg-garden-1889.jpg')

print("The type of this input is {}".format(type(image)))
print("Shape: {}".format(image.shape))
plt.imshow(image)

The type of this input is <class 'NoneType'>


[ WARN:0@7002.883] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('https://uploads3.wikiart.org/00123/images/charles-courtney-curran/in-the-luxembourg-garden-1889.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imshow(gray_image, cmap='gray')

In [ ]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [ ]:
def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
def get_colors(image, number_of_colors, show_chart):
    
    modified_image = image.reshape(image.shape[0]*image.shape[1], 3)
    
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)
    
    counts = Counter(labels)
    # sort to ensure correct color percentage
    counts = dict(sorted(counts.items()))
    
    center_colors = clf.cluster_centers_
    # We get ordered colors by iterating through the keys
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
    rgb_colors = [ordered_colors[i] for i in counts.keys()]

    if (show_chart):
        plt.figure(figsize = (8, 6))
        plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)
    
    return rgb_colors

In [ ]:
get_colors(get_image("2011-01580.jpg"), 7, True)

# Search with colour

In [ ]:
IMAGE_DIRECTORY = 'images'
COLORS = {
    'GREEN': [0, 128, 0],
    'BLUE': [0, 0, 128],
    'YELLOW': [255, 255, 0]
}
images = []

for file in os.listdir(IMAGE_DIRECTORY):
    if not file.startswith('.'):
        images.append(get_image(os.path.join(IMAGE_DIRECTORY, file)))

In [ ]:
plt.figure(figsize=(20, 10))
for i in range(len(images)):
    plt.subplot(1, len(images), i+1)
    plt.imshow(images[i])

In [ ]:
def match_image_by_color(image, color, threshold = 60, number_of_colors = 10): 
    
    image_colors = get_colors(image, number_of_colors, False)
    selected_color = rgb2lab(np.uint8(np.asarray([[color]])))

    select_image = False
    for i in range(number_of_colors):
        curr_color = rgb2lab(np.uint8(np.asarray([[image_colors[i]]])))
        diff = deltaE_cie76(selected_color, curr_color)
        if (diff < threshold):
            select_image = True
    
    return select_image

In [ ]:
def show_selected_images(images, color, threshold, colors_to_match):
    index = 1
    
    for i in range(len(images)):
        selected = match_image_by_color(images[i],
                                        color,
                                        threshold,
                                        colors_to_match)
        if (selected):
            plt.subplot(1, 5, index)
            plt.imshow(images[i])
            index += 1

In [ ]:
# Search for GREEN
plt.figure(figsize = (20, 10))
show_selected_images(images, COLORS['GREEN'], 60, 5)

In [ ]:
# Search for BLUE
plt.figure(figsize = (20, 10))
show_selected_images(images, COLORS['BLUE'], 60, 5)

In [ ]:
# Search for YELLOW
plt.figure(figsize = (20, 10))
show_selected_images(images, COLORS['YELLOW'], 60, 5)